In [1]:
#!pip download lifelines
#%pip install input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
#%pip install input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
#%pip install input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
#%pip install input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
#%pip install input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from scipy.stats import rankdata 
import numpy as np
from tqdm import tqdm

import lightgbm as lgb
from lightgbm import LGBMRegressor

from scipy.stats import rankdata 
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import PolynomialFeatures
from lifelines import KaplanMeierFitter, NelsonAalenFitter
from sklearn.preprocessing import StandardScaler, LabelEncoder
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier

from catboost import CatBoostRegressor, CatBoostClassifier
import catboost as cb

from metric import score
import json

In [ ]:
# set analysis output directory
def create_output_directory(output_path):
    """Create the output directory if it doesn't exist and set plotting style."""
    os.makedirs(output_path, exist_ok=True)
    return output_path

output_path = 'working/analysis'
create_output_directory(output_path)

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

test = pd.read_csv("input/data/test.csv")
print("Test shape:", test.shape )

train = pd.read_csv("input/data/train.csv")
print("Train shape:",train.shape)
train.head()

In [ ]:
# Train targets
plt.hist(train.loc[train.efs==1,"efs_time"],bins=100,label="efs=1, Yes Event")
plt.hist(train.loc[train.efs==0,"efs_time"],bins=100,label="efs=0, Maybe Event")
plt.xlabel("Time of Observation, efs_time")
plt.ylabel("Density")
plt.title("Times of Observation. Either time to event, or time observed without event.")
plt.legend()
plt.savefig(f'{output_path}times_of_observation.png')
plt.show()

In [ ]:
# Transform Two Targets into One Target with KaplanMeier
def transform_survival_probability(df, time_col='efs_time', event_col='efs'):
    kmf = KaplanMeierFitter()
    kmf.fit(df[time_col], df[event_col])
    y = kmf.survival_function_at_times(df[time_col]).values
    return y
train["y"] = transform_survival_probability(train, time_col='efs_time', event_col='efs')

plt.hist(train.loc[train.efs==1,"y"],bins=100,label="efs=1, Yes Event")
plt.hist(train.loc[train.efs==0,"y"],bins=100,label="efs=0, Maybe Event")
plt.xlabel("Transformed Target y")
plt.ylabel("Density")
plt.title("KaplanMeier Transformed Target y using both efs and efs_time.")
plt.legend()
plt.savefig(f'{output_path}kaplanmeier_transformed_target_y.png')
plt.show()

In [ ]:
# missing values heatmap
def plot_missing_values_heatmap(df, output_path):
    plt.figure(figsize=(16, 8))
    sns.heatmap(df.isnull(), yticklabels=False, cbar=True, cmap='viridis')
    plt.title('Missing Values Heatmap')
    plt.tight_layout()
    plt.savefig(f'{output_path}/missing_values_heatmap.png')
    plt.show()
    plt.close()

plot_missing_values_heatmap(train, output_path)

In [ ]:
# missing value percentages
def plot_missing_values_bars(df, output_path):
    plt.figure(figsize=(16, 10))
    missing_percentages = (df.isnull().sum() / len(df) * 100).sort_values(ascending=True)
    sns.barplot(x=missing_percentages.values, y=missing_percentages.index)
    plt.title('Percentage of Missing Values by Column')
    plt.xlabel('Percentage Missing')
    plt.tight_layout()
    plt.savefig(f'{output_path}/missing_values_percentage.png')
    plt.show()
    plt.close()

plot_missing_values_bars(train, output_path)

In [ ]:
# numerical distributions
def plot_numerical_distributions(df, output_path):
    numerical_cols = df.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns
    
    # Create progress bar for numerical distributions
    for col in tqdm(numerical_cols, desc="Creating distribution plots"):
        plt.figure(figsize=(10, 6))
        
        # Create subplot with histogram and kde
        sns.histplot(data=df, x=col, kde=True)
        plt.title(f'Distribution of {col}')
        plt.xlabel(col)
        plt.ylabel('Count')
        
        # Add statistical annotations
        stats_text = f'Mean: {df[col].mean():.2f}\n'
        stats_text += f'Median: {df[col].median():.2f}\n'
        stats_text += f'Std: {df[col].std():.2f}'
        plt.text(0.95, 0.95, stats_text,
                transform=plt.gca().transAxes,
                verticalalignment='top',
                horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.tight_layout()
        plt.savefig(f'{output_path}/distribution_{col}.png')
        plt.close()

plot_numerical_distributions(train, output_path)

In [ ]:
# correlation matrix
def plot_correlation_matrix(df, output_path):
    numerical_cols = df.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns
    
    if len(numerical_cols) > 1:
        plt.figure(figsize=(24, 16))
        correlation_matrix = df[numerical_cols].corr()
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
        plt.title('Correlation Matrix')
        plt.tight_layout()
        plt.savefig(f'{output_path}/correlation_matrix.png')
        plt.show()
        plt.close()

plot_correlation_matrix(train, output_path)

In [ ]:
# categorical distributions
def plot_categorical_distributions(df, output_path):
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    
    for col in tqdm(categorical_cols, desc="Creating categorical plots"):
        if df[col].nunique() < 30:
            plt.figure(figsize=(12, 6))
            value_counts = df[col].value_counts()
            sns.barplot(x=value_counts.index, y=value_counts.values)
            plt.title(f'Distribution of {col}')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(f'{output_path}/categorical_{col}.png')
            plt.close()

plot_categorical_distributions(train, output_path)

In [ ]:
RMV = ["ID","efs","efs_time","y"]
FEATURES = [c for c in train.columns if not c in RMV]
print(f"Number of Features: {len(FEATURES)} FEATURES: {FEATURES}")

In [ ]:
CATS = []
for c in FEATURES:
    if train[c].dtype=="object":
        CATS.append(c)
        train[c] = train[c].fillna("Missing")
        test[c] = test[c].fillna("Missing")
print(f"In these features, there are {len(CATS)} CATEGORICAL FEATURES: {CATS}")

In [14]:
# Feature Engineering
def feature_engineering(df):   
    # Create age bins in 5-year intervals
    median_age = df['age_at_hct'].median()
    df['age_at_hct'] = df['age_at_hct'].replace([np.inf, -np.inf, '', None], np.nan).fillna(median_age)
    df['age_bin'] = pd.cut(df['age_at_hct'], 
                          bins=range(0, 95, 5),  # From 0 to 75 in steps of 5
                          labels=[f'{i}-{i+4}' for i in range(0, 90, 5)],
                          include_lowest=True)
    
    # commorbity by age at hct
    df['comorbidity_age'] = df['comorbidity_score'] * df['age_at_hct']
    
    df['age_bin_race'] = (df['age_bin'].astype(str) + '_' + 
                      df['race_group'].astype(str)).astype('category')

    # age x cyto
    df['cyto_age'] = (df['cyto_score_detail'].astype(str) + '_' + df['age_at_hct'].astype(str)).astype('category')

    # Concatenate graft_type and prod_type
    df['graft_prod'] = (df['graft_type'].astype(str) + '_' + 
                    df['prod_type'].astype(str)).astype('category')
    
    # Concatenate age bin and pulm severe
    df['age_bin_pulm_severe'] = (df['age_bin'].astype(str) + '_' + 
                    df['pulm_severe'].astype(str)).astype('category')
    
    # Concatenate age bin, race_group, and dri score
    df['age_bin_dri'] = (df['age_bin'].astype(str) + '_' + 
                    df['dri_score'].astype(str)).astype('category')

    # hla high mean
    df['hla_high_res_mean'] = df[['hla_high_res_8', 'hla_high_res_10', 'hla_high_res_6']].mean(axis=1)

    # hla low mean
    df['hla_low_res_mean'] = df[['hla_low_res_8', 'hla_low_res_10', 'hla_low_res_6']].mean(axis=1)

    # ration of hla high and hla low
    df['hla_ratio_res_highlow'] = (df['hla_high_res_mean']+1)/(df['hla_low_res_mean']+1)

    # ration of hla low and hla high
    df['hla_ratio_res_lowhigh'] = (df['hla_low_res_mean']+1)/(df['hla_high_res_mean']+1)

    # age functions
    df['donor_by_age_at_hct'] = (df['donor_age']/df['age_at_hct'])
    df['comorbidity_score_by_age_at_hct'] = (df['comorbidity_score']/df['age_at_hct'])

    # match drb
    df['hla_match_drb1_mean'] = df[['hla_match_drb1_high', 'hla_match_drb1_low']].mean(axis=1)

    # match dqb
    df['hla_match_dqb1_mean'] = df[['hla_match_dqb1_high', 'hla_match_dqb1_low']].mean(axis=1)

    # additional ratios
    df['hla_high_low_ratio'] = (df['hla_high_res_mean'] + 1) / (df['hla_low_res_mean'] + 1)
    df['drb1_dqb1_ratio'] = (df['hla_match_drb1_mean'] + 1) / (df['hla_match_dqb1_mean'] + 1)

    # difference in features
    df['high_low_diff'] = df['hla_high_res_mean'] - df['hla_low_res_mean']
    df['drb1_dqb1_diff'] = df['hla_match_drb1_mean'] - df['hla_match_dqb1_mean']

    # statistical aggregations
    df['hla_mean'] = df[['hla_high_res_mean', 'hla_low_res_mean', 'hla_match_drb1_mean', 'hla_match_dqb1_mean']].mean(axis=1)
    df['hla_std'] = df[['hla_high_res_mean', 'hla_low_res_mean', 'hla_match_drb1_mean', 'hla_match_dqb1_mean']].std(axis=1)
    df['hla_max'] = df[['hla_high_res_mean', 'hla_low_res_mean', 'hla_match_drb1_mean', 'hla_match_dqb1_mean']].max(axis=1)
    df['hla_min'] = df[['hla_high_res_mean', 'hla_low_res_mean', 'hla_match_drb1_mean', 'hla_match_dqb1_mean']].min(axis=1)

    # interaction terms
    df['drb1_high_interaction'] = df['hla_match_drb1_mean'] * df['hla_high_res_mean']
    df['dqb1_low_interaction'] = df['hla_match_dqb1_mean'] * df['hla_low_res_mean']

    # with or without TBI
    df['with_tbi'] = np.where((df['tbi_status']).astype(str) == 'No TBI', 'no', 'yes')

    # Create donor sex feature
    df['sex_donor'] = (df['sex_match']).astype(str).str[0]

    # Create recipient sex feature
    df['sex_recipient'] = (df['sex_match']).astype(str).str[2]

    # Main drug presence
    df['has_FK'] = (df['gvhd_proph']).astype(str).str.contains('FK', na=False).astype(int)
    df['has_MMF'] = (df['gvhd_proph']).astype(str).str.contains('MMF', na=False).astype(int)
    df['has_MTX'] = (df['gvhd_proph']).astype(str).str.contains('MTX', na=False).astype(int)
    df['has_CSA'] = (df['gvhd_proph']).astype(str).str.contains('CSA', na=False).astype(int)
    df['has_cyclophosphamide'] = (df['gvhd_proph']).astype(str).str.contains('Cyclophosphamide', na=False).astype(int)

    # Check for combination therapy
    df['has_combination'] = (df['gvhd_proph']).astype(str).str.contains('\+', na=False).astype(int)

    # Count number of agents (approximate by counting '+' signs)
    df['n_agents'] = (df['gvhd_proph']).astype(str).str.count('\+').add(1)

    # Complex vs Simple regimen
    df['is_complex'] = (df['gvhd_proph']).astype(str).str.contains('others', na=False).astype(int)

    # Depletion-based therapy
    df['is_depletion_based'] = (df['gvhd_proph']).astype(str).str.contains('TDEPLETION|CDselect', na=False).astype(int)

    # Monotherapy flag
    df['is_monotherapy'] = (df['gvhd_proph']).astype(str).str.contains('alone', na=False).astype(int)

    # No prophylaxis flag
    df['no_prophylaxis'] = ((df['gvhd_proph']).astype(str) == 'No GvHD Prophylaxis').astype(int)

    def get_primary_agent(x):
        if pd.isna(x):
            return 'Unknown'
        elif 'FK' in x:
            return 'FK-based'
        elif 'CSA' in x:
            return 'CSA-based'
        elif 'Cyclophosphamide' in x:
            return 'Cyclophosphamide-based'
        elif 'TDEPLETION' in x or 'CDselect' in x:
            return 'Depletion-based'
        else:
            return 'Other'

    df['primary_agent'] = (df['gvhd_proph']).astype(str).apply(get_primary_agent)

    # Standard vs Alternative approach
    df['is_standard_approach'] = (df['gvhd_proph']).astype(str).str.contains('FK\+ MMF|FK\+ MTX|CSA \+ MTX', na=False).astype(int)

    # Experimental/Other
    df['is_experimental'] = ((df['gvhd_proph']).astype(str) == 'Other GVHD Prophylaxis').astype(int)

    # Combine with other relevant features
    df['FK_MMF_interaction'] = df['has_FK'] * df['has_MMF']
    df['CSA_MTX_interaction'] = df['has_CSA'] * df['has_MTX']

    return df

train = feature_engineering(train)
train = train.dropna(thresh=10, axis=0)
test = feature_engineering(test)

In [15]:
FEATURES = [c for c in train.columns if not c in RMV]

In [16]:
# Get categorical columns from train features
CATS = train[FEATURES].select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
# correlation matrixn after feature engineering
def plot_correlation_matrix(df, output_path):
    numerical_cols = df.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns
    
    if len(numerical_cols) > 1:
        plt.figure(figsize=(42, 38))
        correlation_matrix = df[numerical_cols].corr()
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
        plt.title('Correlation Matrix After Feature Engineering')
        plt.tight_layout()
        plt.savefig(f'{output_path}/correlation_matrix_feature_engineering.png')
        plt.show()
        plt.close()

plot_correlation_matrix(train, output_path)

In [ ]:
combined = pd.concat([train,test],axis=0,ignore_index=True)
#print("Combined data shape:", combined.shape )

# LABEL ENCODE CATEGORICAL FEATURES
print("We LABEL ENCODE the CATEGORICAL FEATURES: ",end="")
for c in FEATURES:

    # LABEL ENCODE CATEGORICAL AND CONVERT TO INT32 CATEGORY
    if c in CATS:
        print(f"{c}, ",end="")
        combined[c],_ = combined[c].factorize()
        combined[c] -= combined[c].min()
        combined[c] = combined[c].astype("int32")
        combined[c] = combined[c].astype("category")
        
    # REDUCE PRECISION OF NUMERICAL TO 32BIT TO SAVE MEMORY
    else:
        if combined[c].dtype=="float64":
            combined[c] = combined[c].astype("float32")
        if combined[c].dtype=="int64":
            combined[c] = combined[c].astype("int32")
    
train = combined.iloc[:len(train)].copy()
test = combined.iloc[len(train):].reset_index(drop=True).copy()

In [ ]:
for col in CATS:
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

for col in CATS:
    # Ensure categories are coded as integers starting from 0
    train[col] = train[col].cat.codes
    test[col] = test[col].cat.codes
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')
    
    # Verify the encoding
    print(f"\nAfter fixing {col}:")
    print(train[col].value_counts())

In [20]:
SEED = 9365

def perform_pca(train, test, n_components=None, random_state=42):
    # Remove rows with NaN values from both datasets
    train = train.dropna()
    test = test.dropna()

    pca = PCA(n_components=n_components, random_state=random_state)
    train_pca = pca.fit_transform(train)
    test_pca = pca.transform(test)
    
    explained_variance_ratio = pca.explained_variance_ratio_
    print(f"Explained variance ratio of the components:\n {explained_variance_ratio}")
    print(np.sum(explained_variance_ratio))
    
    train_pca_df = pd.DataFrame(train_pca, columns=[f'PC_{i+1}' for i in range(train_pca.shape[1])])
    test_pca_df = pd.DataFrame(test_pca, columns=[f'PC_{i+1}' for i in range(test_pca.shape[1])])
    
    return train_pca_df, test_pca_df, pca

In [ ]:
# PCA 
# Extract the numerical columns to be used in the PCA
train_num = train.drop('ID', axis=1)
test_num = test.drop('ID', axis=1)

# Get numeric and categorical columns
numeric_columns = train.select_dtypes(include=['int32', 'float32']).columns
categorical_columns = train.select_dtypes(exclude=['int32', 'float32']).columns

# Split into numeric and categorical dataframes
train_numeric = train_num[numeric_columns]
test_numeric = test_num[numeric_columns]
train_categorical = train[categorical_columns]
test_categorical = test[categorical_columns]

# Scale the numeric columns
scaler = StandardScaler()
train_scaled = pd.DataFrame(
    scaler.fit_transform(train_numeric),
    columns=train_numeric.columns
)
test_scaled = pd.DataFrame(
    scaler.transform(test_numeric),
    columns=test_numeric.columns
)

train_pca, test_pca, pca = perform_pca(train_scaled, test_scaled, n_components=15, random_state=SEED)

# Merge scaled numeric data with categorical data
train_final = pd.concat([train_scaled, train_categorical, train_pca], axis=1)
test_final = pd.concat([test_scaled, test_categorical, test_pca], axis=1)

In [22]:
test = test_final
train = train_final

# add pca columns to features list
FEATURES.extend(train_pca.columns.tolist())

In [ ]:
FEATURES

In [ ]:
train.dtypes

In [ ]:
# XGBoost with KaplanMeier
print("Using XGBoost version",xgb.__version__)

In [26]:
# feature selection
FEATURES = ['dri_score',
    'psych_disturb',
    'cyto_score',
    'diabetes',
    #'hla_match_c_high',
    'hla_high_res_8',
    'tbi_status',
    'arrhythmia',
    'hla_low_res_6',
    'graft_type',
    #'vent_hist',
    'renal_issue',
    'pulm_severe',
    'prim_disease_hct',
    'hla_high_res_6',
    'cmv_status',
    'hla_high_res_10',
    'hla_match_dqb1_high',
    'tce_imm_match',
    'hla_nmdp_6',
    'hla_match_c_low',
    'rituximab',
    'hla_match_drb1_low',
    'hla_match_dqb1_low',
    'prod_type',
    #'cyto_score_detail',
    'conditioning_intensity',
    #'ethnicity',
    'year_hct',
    'obesity',
    'mrd_hct',
    'in_vivo_tcd',
    'tce_match',
    'hla_match_a_high',
    'hepatic_severe',
    'donor_age',
    'prior_tumor',
    'hla_match_b_low',
    'peptic_ulcer',
    'age_at_hct',
    'hla_match_a_low',
    'gvhd_proph',
    'rheum_issue',
    'sex_match',
    'hla_match_b_high',
    'race_group',
    'comorbidity_score',
    'karnofsky_score',
    'hepatic_mild',
    'tce_div_match',
    'donor_related',
    'melphalan_dose',
    'hla_low_res_8',
    'cardiac',
    'hla_match_drb1_high',
    'pulm_moderate',
    'hla_low_res_10',
    #'PC_1',
    #'PC_2',
    #'PC_3',
    #'PC_4',
    #'PC_5',
    #'PC_6',
    #'PC_7',
    #'PC_8',
    #'PC_9',
    #'PC_10',
    #'PC_11',
    #'PC_12',
    #'PC_13',
    #'PC_14',
    #'PC_15',
    'comorbidity_age',
    'age_bin',
    #'cyto_age',
    'graft_prod',
    'age_bin_race',
    'age_bin_pulm_severe',
    'hla_high_res_mean',
    'hla_low_res_mean',
    #'hla_ratio_res_highlow',
    'hla_ratio_res_lowhigh',
    'age_bin_dri',
    'donor_by_age_at_hct',
    'comorbidity_score_by_age_at_hct',
    #'hla_match_drb1_mean',
    'hla_match_dqb1_mean',
    'hla_high_low_ratio',
    'drb1_dqb1_ratio',
    #'high_low_diff',
    'drb1_dqb1_diff',
    'hla_mean',
    'hla_std',
    'hla_max',
    #'hla_min',
    'drb1_high_interaction',
    'dqb1_low_interaction',
    'with_tbi',
    'sex_donor',
    'sex_recipient',
    'has_FK',
    'has_MMF',
    'has_MTX',
    'has_CSA',
    #'has_cyclophosphamide',
    'has_combination',
    'n_agents',
    'is_complex',
    'is_depletion_based',
    'is_monotherapy',
    #'no_prophylaxis',
    'primary_agent',
    'is_standard_approach',
    #'is_experimental',
    #'FK_MMF_interaction',
    'CSA_MTX_interaction'
    ]

# Get categorical columns from train features
CATS = train[FEATURES].select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
%%time
FOLDS = 10
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
    
oof_xgb = np.zeros(len(train))
pred_xgb = np.zeros(len(test))

for i, (train_index, test_index) in enumerate(kf.split(train)):

    print("#"*25)
    print(f"### Fold {i+1}")
    print("#"*25)
    
    x_train = train.loc[train_index,FEATURES].copy()
    y_train = train.loc[train_index,"y"]
    x_valid = train.loc[test_index,FEATURES].copy()
    y_valid = train.loc[test_index,"y"]
    x_test = test[FEATURES].copy()

    model_xgb = XGBRegressor(
        device="cuda",
        max_depth=6,  
        colsample_bytree=0.5,  
        subsample=0.8,  
        n_estimators=2000,  
        learning_rate=0.01,  
        enable_categorical=True,
        min_child_weight=80,
        #early_stopping_rounds=25,
    )
    model_xgb.fit(
        x_train, y_train,
        eval_set=[(x_valid, y_valid)],  
        verbose=500 
    )

    # INFER OOF
    oof_xgb[test_index] = model_xgb.predict(x_valid)
    # INFER TEST
    pred_xgb += model_xgb.predict(x_test)

# COMPUTE AVERAGE TEST PREDS
pred_xgb /= FOLDS

In [ ]:
y_true = train[["ID","efs","efs_time","race_group"]].copy()
y_pred = train[["ID"]].copy()
y_pred["prediction"] = oof_xgb
m = score(y_true.copy(), y_pred.copy(), "ID")
print(f"\nOverall CV for XGBoost KaplanMeier =",m)

In [ ]:
feature_importance = model_xgb.feature_importances_
importance_df = pd.DataFrame({
    "Feature": FEATURES,  
    "Importance": feature_importance
}).sort_values(by="Importance", ascending=False)
plt.figure(figsize=(20, 20))
plt.barh(importance_df["Feature"], importance_df["Importance"])
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("XGBoost KaplanMeier Feature Importance")
plt.gca().invert_yaxis()  # Flip features for better readability
plt.savefig(f'{output_path}xgboost_km_feature_importance.png')
plt.show()

In [ ]:
# CatBoost with KaplanMeier
print("Using CatBoost version",cb.__version__)

In [31]:
# feature selection
FEATURES = ['dri_score',
    'psych_disturb',
    'cyto_score',
    'diabetes',
    #'hla_match_c_high',
    #'hla_high_res_8',
    'tbi_status',
    'arrhythmia',
    #'hla_low_res_6',
    #'graft_type',
    'vent_hist',
    #'renal_issue',
    'pulm_severe',
    'prim_disease_hct',
    #'hla_high_res_6',
    'cmv_status',
    #'hla_high_res_10',
    #'hla_match_dqb1_high',
    'tce_imm_match',
    'hla_nmdp_6',
    #'hla_match_c_low',
    #'rituximab',
    #'hla_match_drb1_low',
    #'hla_match_dqb1_low',
    #'prod_type',
    'cyto_score_detail',
    'conditioning_intensity',
    'ethnicity',
    'year_hct',
    'obesity',
    'mrd_hct',
    'in_vivo_tcd',
    'tce_match',
    'hla_match_a_high',
    'hepatic_severe',
    'donor_age',
    'prior_tumor',
    #'hla_match_b_low',
    #'peptic_ulcer',
    'age_at_hct',
    'hla_match_a_low',
    'gvhd_proph',
    'rheum_issue',
    'sex_match',
    #'hla_match_b_high',
    'race_group',
    'comorbidity_score',
    'karnofsky_score',
    'hepatic_mild',
    'tce_div_match',
    'donor_related',
    'melphalan_dose',
    #'hla_low_res_8',
    'cardiac',
    'hla_match_drb1_high',
    'pulm_moderate',
    #'hla_low_res_10',
    #'PC_1',
    #'PC_2',
    #'PC_3',
    #'PC_4',
    #'PC_5',
    #'PC_6',
    #'PC_7',
    #'PC_8',
    #'PC_9',
    #'PC_10',
    #'PC_11',
    #'PC_12',
    #'PC_13',
    #'PC_14',
    #'PC_15',
    'comorbidity_age',
    'age_bin',
    #'cyto_age',
    'graft_prod',
    'age_bin_race',
    'age_bin_pulm_severe',
    #'hla_high_res_mean',
    'hla_low_res_mean',
    'hla_ratio_res_highlow',
    'hla_ratio_res_lowhigh',
    'age_bin_dri',
    'donor_by_age_at_hct',
    'comorbidity_score_by_age_at_hct',
    #'hla_match_drb1_mean',
    #'hla_match_dqb1_mean',
    #'hla_high_low_ratio',
    'drb1_dqb1_ratio',
    #'high_low_diff',
    #'drb1_dqb1_diff',
    'hla_mean',
    'hla_std',
    #'hla_max',
    #'hla_min',
    'drb1_high_interaction',
    'dqb1_low_interaction',
    'with_tbi',
    'sex_donor',
    #'sex_recipient',
    'has_FK',
    #'has_MMF',
    #'has_MTX',
    #'has_CSA',
    #'has_cyclophosphamide',
    #'has_combination',
    #'n_agents',
    #'is_complex',
    #'is_depletion_based',
    #'is_monotherapy',
    #'no_prophylaxis',
    'primary_agent',
    #'is_standard_approach',
    #'is_experimental',
    #'FK_MMF_interaction',
    #'CSA_MTX_interaction'
    ]

# Get categorical columns from train features
CATS = train[FEATURES].select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
%%time
FOLDS = 10
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
    
oof_cat = np.zeros(len(train))
pred_cat = np.zeros(len(test))

for i, (train_index, test_index) in enumerate(kf.split(train)):

    print("#"*25)
    print(f"### Fold {i+1}")
    print("#"*25)
    
    x_train = train.loc[train_index,FEATURES].copy()
    y_train = train.loc[train_index,"y"]
    x_valid = train.loc[test_index,FEATURES].copy()
    y_valid = train.loc[test_index,"y"]
    x_test = test[FEATURES].copy()

    model_cat = CatBoostRegressor(
        task_type="GPU",  
        learning_rate=0.01,    
        grow_policy='Lossguide',
        depth=6,                    # Control tree depth (default=6)
        min_data_in_leaf=1,        # Minimum number of training samples in a leaf
        l2_leaf_reg=3.0,           # L2 regularization coefficient
        random_strength=1,          # Random score coefficient for splitting
        n_estimators=2000,
        bootstrap_type='Bernoulli', # Try 'Bayesian', 'Bernoulli', 'MVS'
        subsample=0.8,             # Sample rate for bootstrap
        early_stopping_rounds=50
    )
    model_cat.fit(x_train,y_train,
              eval_set=(x_valid, y_valid),
              cat_features=CATS,
              verbose=250)

    # INFER OOF
    oof_cat[test_index] = model_cat.predict(x_valid)
    # INFER TEST
    pred_cat += model_cat.predict(x_test)

# COMPUTE AVERAGE TEST PREDS
pred_cat /= FOLDS

In [ ]:
y_true = train[["ID","efs","efs_time","race_group"]].copy()
y_pred = train[["ID"]].copy()
y_pred["prediction"] = oof_cat
m = score(y_true.copy(), y_pred.copy(), "ID")
print(f"\nOverall CV for CatBoost KaplanMeier =",m)

In [ ]:
feature_importance = model_cat.get_feature_importance()
importance_df = pd.DataFrame({
    "Feature": FEATURES, 
    "Importance": feature_importance
}).sort_values(by="Importance", ascending=False)
plt.figure(figsize=(20, 20))
plt.barh(importance_df["Feature"], importance_df["Importance"])
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("CatBoost KaplanMeier Feature Importance")
plt.gca().invert_yaxis()  # Flip features for better readability
plt.savefig(f'{output_path}catboost_km_feature_importance.png')
plt.show()

In [ ]:
# LightGBM with KaplanMeier
print("Using LightGBM version",lgb.__version__)

In [36]:
# feature selection
FEATURES = ['dri_score',
    'psych_disturb',
    'cyto_score',
    'diabetes',
    'hla_match_c_high',
    #'hla_high_res_8',
    'tbi_status',
    'arrhythmia',
    #'hla_low_res_6',
    #'graft_type',
    'vent_hist',
    #'renal_issue',
    #'pulm_severe',
    'prim_disease_hct',
    #'hla_high_res_6',
    'cmv_status',
    #'hla_high_res_10',
    #'hla_match_dqb1_high',
    'tce_imm_match',
    'hla_nmdp_6',
    'hla_match_c_low',
    #'rituximab',
    #'hla_match_drb1_low',
    #'hla_match_dqb1_low',
    #'prod_type',
    'cyto_score_detail',
    'conditioning_intensity',
    'ethnicity',
    'year_hct',
    'obesity',
    'mrd_hct',
    'in_vivo_tcd',
    'tce_match',
    'hla_match_a_high',
    'hepatic_severe',
    'donor_age',
    'prior_tumor',
    'hla_match_b_low',
    #'peptic_ulcer',
    #'age_at_hct',
    'hla_match_a_low',
    'gvhd_proph',
    'rheum_issue',
    'sex_match',
    #'hla_match_b_high',
    'race_group',
    'comorbidity_score',
    'karnofsky_score',
    'hepatic_mild',
    'tce_div_match',
    'donor_related',
    'melphalan_dose',
    #'hla_low_res_8',
    'cardiac',
    #'hla_match_drb1_high',
    #'pulm_moderate',
    #'hla_low_res_10',
    #'PC_1',
    #'PC_2',
    #'PC_3',
    #'PC_4',
    #'PC_5',
    #'PC_6',
    #'PC_7',
    #'PC_8',
    #'PC_9',
    #'PC_10',
    #'PC_11',
    #'PC_12',
    #'PC_13',
    #'PC_14',
    #'PC_15',
    'comorbidity_age',
    'age_bin',
    #'cyto_age',
    'graft_prod',
    'age_bin_race',
    'age_bin_pulm_severe',
    'hla_high_res_mean',
    'hla_low_res_mean',
    'hla_ratio_res_highlow',
    'hla_ratio_res_lowhigh',
    'age_bin_dri',
    'donor_by_age_at_hct',
    'comorbidity_score_by_age_at_hct',
    #'hla_match_drb1_mean',
    #'hla_match_dqb1_mean',
    #'hla_high_low_ratio',
    'drb1_dqb1_ratio',
    #'high_low_diff',
    #'drb1_dqb1_diff',
    'hla_mean',
    'hla_std',
    #'hla_max',
    #'hla_min',
    'drb1_high_interaction',
    'dqb1_low_interaction',
    #'with_tbi',
    #'sex_donor',
    #'sex_recipient',
    #'has_FK',
    #'has_MMF',
    #'has_MTX',
    #'has_CSA',
    #'has_cyclophosphamide',
    #'has_combination',
    #'n_agents',
    #'is_complex',
    #'is_depletion_based',
    #'is_monotherapy',
    #'no_prophylaxis',
    #'primary_agent',
    #'is_standard_approach',
    #'is_experimental',
    #'FK_MMF_interaction',
    #'CSA_MTX_interaction'
    ]

# Get categorical columns from train features
CATS = train[FEATURES].select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
FOLDS = 10
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
    
oof_lgb = np.zeros(len(train))
pred_lgb = np.zeros(len(test))

for i, (train_index, test_index) in enumerate(kf.split(train)):

    print("#"*25)
    print(f"### Fold {i+1}")
    print("#"*25)
    
    x_train = train.loc[train_index,FEATURES].copy()
    y_train = train.loc[train_index,"y"]    
    x_valid = train.loc[test_index,FEATURES].copy()
    y_valid = train.loc[test_index,"y"]
    x_test = test[FEATURES].copy()

    model_lgb = LGBMRegressor(
        device="gpu",
        num_leaves=31,
        max_depth=3,
        colsample_bytree=0.4,
        subsample=0.8,
        n_estimators=2500,
        learning_rate=0.01,
        objective="regression",
        verbose=-1,
        max_bin=63,  
        gpu_use_dp=True
    )
    model_lgb.fit(
        x_train, y_train,
        eval_set=[(x_valid, y_valid)],
    )
    
    # INFER OOF
    oof_lgb[test_index] = model_lgb.predict(x_valid)
    # INFER TEST
    pred_lgb += model_lgb.predict(x_test)

# COMPUTE AVERAGE TEST PREDS
pred_lgb /= FOLDS

In [ ]:
y_true = train[["ID","efs","efs_time","race_group"]].copy()
y_pred = train[["ID"]].copy()
y_pred["prediction"] = oof_lgb
m = score(y_true.copy(), y_pred.copy(), "ID")
print(f"\nOverall CV for LightGBM KaplanMeier =",m)

In [ ]:
feature_importance = model_lgb.feature_importances_ 
importance_df = pd.DataFrame({
    "Feature": FEATURES,
    "Importance": feature_importance
}).sort_values(by="Importance", ascending=False)
plt.figure(figsize=(20, 20))
plt.barh(importance_df["Feature"], importance_df["Importance"], color='skyblue')
plt.xlabel("Importance (Gain)")
plt.ylabel("Feature")
plt.title("LightGBM KaplanMeier Feature Importance")
plt.gca().invert_yaxis()  # Flip features for better readability
plt.savefig(f'{output_path}lightgbm_km_feature_importance.png')
plt.show()

In [40]:
# XGBoost with Survival:Cox
# SURVIVAL COX NEEDS THIS TARGET (TO DIGEST EFS AND EFS_TIME)
train["efs_time2"] = train.efs_time.copy()
train.loc[train.efs==0,"efs_time2"] *= -1

In [41]:
# feature selection
FEATURES = ['dri_score',
    'psych_disturb',
    'cyto_score',
    'diabetes',
    'hla_match_c_high',
    #'hla_high_res_8',
    'tbi_status',
    'arrhythmia',
    'hla_low_res_6',
    'graft_type',
    'vent_hist',
    'renal_issue',
    'pulm_severe',
    'prim_disease_hct',
    #'hla_high_res_6',
    'cmv_status',
    'hla_high_res_10',
    'hla_match_dqb1_high',
    'tce_imm_match',
    'hla_nmdp_6',
    'hla_match_c_low',
    'rituximab',
    'hla_match_drb1_low',
    'hla_match_dqb1_low',
    'prod_type',
    'cyto_score_detail',
    'conditioning_intensity',
    'ethnicity',
    'year_hct',
    'obesity',
    'mrd_hct',
    'in_vivo_tcd',
    'tce_match',
    'hla_match_a_high',
    'hepatic_severe',
    'donor_age',
    'prior_tumor',
    'hla_match_b_low',
    #'peptic_ulcer',
    'age_at_hct',
    'hla_match_a_low',
    'gvhd_proph',
    'rheum_issue',
    'sex_match',
    #'hla_match_b_high',
    'race_group',
    'comorbidity_score',
    'karnofsky_score',
    'hepatic_mild',
    'tce_div_match',
    'donor_related',
    'melphalan_dose',
    'hla_low_res_8',
    'cardiac',
    'hla_match_drb1_high',
    'pulm_moderate',
    'hla_low_res_10',
    #'PC_1',
    #'PC_2',
    #'PC_3',
    #'PC_4',
    #'PC_5',
    #'PC_6',
    #'PC_7',
    #'PC_8',
    #'PC_9',
    #'PC_10',
    #'PC_11',
    #'PC_12',
    #'PC_13',
    #'PC_14',
    #'PC_15',
    'comorbidity_age',
    'age_bin',
    #'cyto_age',
    'graft_prod',
    'age_bin_race',
    'age_bin_pulm_severe',
    'hla_high_res_mean',
    'hla_low_res_mean',
    'hla_ratio_res_highlow',
    'hla_ratio_res_lowhigh',
    'age_bin_dri',
    'donor_by_age_at_hct',
    'comorbidity_score_by_age_at_hct',
    'hla_match_drb1_mean',
    'hla_match_dqb1_mean',
    #'hla_high_low_ratio',
    'drb1_dqb1_ratio',
    'high_low_diff',
    'drb1_dqb1_diff',
    'hla_mean',
    'hla_std',
    'hla_max',
    'hla_min',
    'drb1_high_interaction',
    'dqb1_low_interaction',
    'with_tbi',
    #'sex_donor',
    #'sex_recipient',
    'has_FK',
    'has_MMF',
    #'has_MTX',
    #'has_CSA',
    'has_cyclophosphamide',
    #'has_combination',
    'n_agents',
    #'is_complex',
    #'is_depletion_based',
    #'is_monotherapy',
    #'no_prophylaxis',
    'primary_agent',
    'is_standard_approach',
    #'is_experimental',
    'FK_MMF_interaction',
    #'CSA_MTX_interaction'
    ]

# Get categorical columns from train features
CATS = train[FEATURES].select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
FOLDS = 10
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
    
oof_xgb_cox = np.zeros(len(train))
pred_xgb_cox = np.zeros(len(test))

for i, (train_index, test_index) in enumerate(kf.split(train)):

    print("#"*25)
    print(f"### Fold {i+1}")
    print("#"*25)
    
    x_train = train.loc[train_index,FEATURES].copy()
    y_train = train.loc[train_index,"efs_time2"]    
    x_valid = train.loc[test_index,FEATURES].copy()
    y_valid = train.loc[test_index,"efs_time2"]
    x_test = test[FEATURES].copy()

    model_xgb_cox = XGBRegressor(
        device="cuda",
        max_depth=3,  
        colsample_bytree=0.5,  
        subsample=0.8,  
        n_estimators=2000,  
        learning_rate=0.02,  
        enable_categorical=True,
        min_child_weight=80,
        objective='survival:cox',
        eval_metric='cox-nloglik',
    )
    model_xgb_cox.fit(
        x_train, y_train,
        eval_set=[(x_valid, y_valid)],  
        verbose=500  
    )
    
    # INFER OOF
    oof_xgb_cox[test_index] = model_xgb_cox.predict(x_valid)
    # INFER TEST
    pred_xgb_cox += model_xgb_cox.predict(x_test)

# COMPUTE AVERAGE TEST PREDS
pred_xgb_cox /= FOLDS

In [ ]:
y_true = train[["ID","efs","efs_time","race_group"]].copy()
y_pred = train[["ID"]].copy()
y_pred["prediction"] = oof_xgb_cox
m = score(y_true.copy(), y_pred.copy(), "ID")
print(f"\nOverall CV for XGBoost Survival:Cox =",m)

In [ ]:
feature_importance = model_xgb_cox.feature_importances_
importance_df = pd.DataFrame({
    "Feature": FEATURES,  # Replace FEATURES with your list of feature names
    "Importance": feature_importance
}).sort_values(by="Importance", ascending=False)
plt.figure(figsize=(20, 20))
plt.barh(importance_df["Feature"], importance_df["Importance"])
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("XGBoost Survival:Cox Feature Importance")
plt.gca().invert_yaxis()  # Flip features for better readability
plt.savefig(f'{output_path}xgboost_cox_feature_importance.png')
plt.show()

In [45]:
# CatBoost with Survival:Cox
# feature selection
FEATURES = ['dri_score',
    'psych_disturb',
    'cyto_score',
    'diabetes',
    #'hla_match_c_high',
    #'hla_high_res_8',
    'tbi_status',
    'arrhythmia',
    #'hla_low_res_6',
    #'graft_type',
    'vent_hist',
    #'renal_issue',
    'pulm_severe',
    'prim_disease_hct',
    #'hla_high_res_6',
    'cmv_status',
    #'hla_high_res_10',
    'hla_match_dqb1_high',
    'tce_imm_match',
    'hla_nmdp_6',
    'hla_match_c_low',
    #'rituximab',
    #'hla_match_drb1_low',
    'hla_match_dqb1_low',
    #'prod_type',
    'cyto_score_detail',
    'conditioning_intensity',
    'ethnicity',
    'year_hct',
    'obesity',
    'mrd_hct',
    'in_vivo_tcd',
    'tce_match',
    'hla_match_a_high',
    'hepatic_severe',
    'donor_age',
    'prior_tumor',
    'hla_match_b_low',
    'peptic_ulcer',
    'age_at_hct',
    'hla_match_a_low',
    'gvhd_proph',
    #'rheum_issue',
    'sex_match',
    #'hla_match_b_high',
    'race_group',
    'comorbidity_score',
    'karnofsky_score',
    'hepatic_mild',
    'tce_div_match',
    'donor_related',
    'melphalan_dose',
    #'hla_low_res_8',
    'cardiac',
    'hla_match_drb1_high',
    'pulm_moderate',
    #'hla_low_res_10',
    #'PC_1',
    #'PC_2',
    #'PC_3',
    #'PC_4',
    #'PC_5',
    #'PC_6',
    #'PC_7',
    #'PC_8',
    #'PC_9',
    #'PC_10',
    #'PC_11',
    #'PC_12',
    #'PC_13',
    #'PC_14',
    #'PC_15',
    'comorbidity_age',
    'age_bin',
    #'cyto_age',
    'graft_prod',
    'age_bin_race',
    'age_bin_pulm_severe',
    #'hla_high_res_mean',
    'hla_low_res_mean',
    #'hla_ratio_res_highlow',
    'hla_ratio_res_lowhigh',
    #'age_bin_dri',
    'donor_by_age_at_hct',
    'comorbidity_score_by_age_at_hct',
    #'hla_match_drb1_mean',
    'hla_match_dqb1_mean',
    #'hla_high_low_ratio',
    'drb1_dqb1_ratio',
    #'high_low_diff',
    #'drb1_dqb1_diff',
    'hla_mean',
    'hla_std',
    #'hla_max',
    #'hla_min',
    #'drb1_high_interaction',
    #'dqb1_low_interaction',
    'with_tbi',
    'sex_donor',
    'sex_recipient',
    'has_FK',
    #'has_MMF',
    #'has_MTX',
    #'has_CSA',
    #'has_cyclophosphamide',
    #'has_combination',
    #'n_agents',
    #'is_complex',
    #'is_depletion_based',
    #'is_monotherapy',
    #'no_prophylaxis',
    'primary_agent',
    #'is_standard_approach',
    #'is_experimental',
    #'FK_MMF_interaction',
    #'CSA_MTX_interaction'
    ]

# Get categorical columns from train features
CATS = train[FEATURES].select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
FOLDS = 10
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
    
oof_cat_cox = np.zeros(len(train))
pred_cat_cox = np.zeros(len(test))

for i, (train_index, test_index) in enumerate(kf.split(train)):

    print("#"*25)
    print(f"### Fold {i+1}")
    print("#"*25)
    
    x_train = train.loc[train_index,FEATURES].copy()
    y_train = train.loc[train_index,"efs_time2"]    
    x_valid = train.loc[test_index,FEATURES].copy()
    y_valid = train.loc[test_index,"efs_time2"]
    x_test = test[FEATURES].copy()

    model_cat_cox = CatBoostRegressor(
        loss_function="Cox",   
        iterations=2000,     
        learning_rate=0.01,  
        grow_policy='SymmetricTree',
        use_best_model=False,
        early_stopping_rounds=100,
        depth=6,
        l2_leaf_reg=5.0
    )
    model_cat_cox.fit(x_train,y_train,
              eval_set=(x_valid, y_valid),
              cat_features=CATS,
              verbose=100)
    
    # INFER OOF
    oof_cat_cox[test_index] = model_cat_cox.predict(x_valid)
    # INFER TEST
    pred_cat_cox += model_cat_cox.predict(x_test)

# COMPUTE AVERAGE TEST PREDS
pred_cat_cox /= FOLDS

In [ ]:
y_true = train[["ID","efs","efs_time","race_group"]].copy()
y_pred = train[["ID"]].copy()
y_pred["prediction"] = oof_cat_cox
m = score(y_true.copy(), y_pred.copy(), "ID")
print(f"\nOverall CV for CatBoost Survival:Cox =",m)

In [ ]:
feature_importance = model_cat_cox.get_feature_importance()
importance_df = pd.DataFrame({
    "Feature": FEATURES, 
    "Importance": feature_importance
}).sort_values(by="Importance", ascending=False)
plt.figure(figsize=(20, 20))
plt.barh(importance_df["Feature"], importance_df["Importance"])
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("CatBoost Survival:Cox Feature Importance")
plt.gca().invert_yaxis()  # Flip features for better readability
plt.savefig(f'{output_path}catboost_cox_feature_importance.png')
plt.show()

In [ ]:
# Ensemble CAT and XGB and LGB
y_true = train[["ID","efs","efs_time","race_group"]].copy()
y_pred = train[["ID"]].copy()
y_pred["prediction"] = rankdata(oof_xgb) + rankdata(oof_cat) + rankdata(oof_lgb)\
                     + rankdata(oof_xgb_cox) + rankdata(oof_cat_cox)
m = score(y_true.copy(), y_pred.copy(), "ID")
print(f"\nOverall CV for Ensemble =",m)

In [ ]:
sub = pd.read_csv("input/data/sample_submission.csv")

# Print individual model rankings with np.set_printoptions for full output
np.set_printoptions(threshold=np.inf)

# Combine rankings
sub.prediction = rankdata(pred_xgb) + rankdata(pred_cat) + rankdata(pred_lgb)\
                    + rankdata(pred_xgb_cox) + rankdata(pred_cat_cox)

print("\nFinal ensemble scores:")
print(sub.prediction)

sub.to_csv("submission.csv", index=False)
print("\nSub shape:", sub.shape)
sub.head()